## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

## Лабораторная работа 3

In [1]:
!pip install line_profiler

In [10]:
import pandas as pd
import numpy as np

In [11]:
recipes = pd.read_csv('recipes_sample.csv')
reviews = pd.read_csv('reviews_sample.csv')
reviews['date'] = pd.to_datetime(reviews['date']) #str to datetime format

In [12]:
#recipes

In [13]:
#reviews

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [14]:
import time

In [15]:
#A
def mean_rating_A(df):
    total = 0 
    count = 0 
    for _, row in df.iterrows(): #iterrows() - итерация в виде пар (индекс, серия)
        if row['date'].year == 2010:
            total += row['rating']
            count += 1
    return total/count

start_time = time.time() 
mean_rating_A(reviews)         
end_time = time.time()    

execution_time = end_time - start_time
print("Время выполнения функции:", execution_time, "секунд")
mean_rating_A(reviews)

Время выполнения функции: 4.137767553329468 секунд


4.4544402182900615

In [16]:
#B
reviews_2010 = reviews[reviews["date"].dt.year == 2010]

In [17]:
def mean_rating_B(df):
    total = 0
    count = 0
    for _, row in df.iterrows():
        total += row["rating"]
        count += 1
    return total / count

start_time = time.time() 
mean_rating_B(reviews)         
end_time = time.time()    

execution_time = end_time - start_time
print("Время выполнения функции:", execution_time, "секунд")
mean_rating_B(reviews_2010)

Время выполнения функции: 4.090382814407349 секунд


4.4544402182900615

In [18]:
#C
def mean_rating_C(df):
    return df['rating'].mean()

start_time = time.time() 
mean_rating_C(reviews)         
end_time = time.time()    

execution_time = end_time - start_time
print("Время выполнения функции:", execution_time, "секунд")
mean_rating_C(reviews_2010)

Время выполнения функции: 0.0 секунд


4.4544402182900615

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [19]:
!pip install line_profiler

In [20]:
from line_profiler import LineProfiler
lp = LineProfiler()
lp_wrapper_A = lp(mean_rating_A)
lp_wrapper_B = lp(mean_rating_B)
lp_wrapper_C = lp(mean_rating_C)

lp_wrapper_A(reviews)
lp_wrapper_B(reviews_2010)
lp_wrapper_C(reviews_2010)
lp.print_stats()

Timer unit: 1e-07 s

Total time: 14.0748 s

Could not find file C:\Users\222844\AppData\Local\Temp/ipykernel_9980/1468465748.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           
     3         1          5.0      5.0      0.0  
     4         1          3.0      3.0      0.0  
     5    126696  124287620.0    981.0     88.3  
     6                                           
     7    114602   15023941.0    131.1     10.7  
     8     12094    1398805.0    115.7      1.0  
     9     12094      37440.0      3.1      0.0  
    10         1          6.0      6.0      0.0  

Total time: 0.0001631 s

Could not find file C:\Users\222844\AppData\Local\Temp/ipykernel_9980/3517253749.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continui

In [21]:
#оптимизированная ф-ия Б
def mean_rating_newB(df):
    total = reviews_2010["rating"].sum()
    count = reviews_2010["rating"].count()
    return total/count
start_time = time.time() 
mean_rating_newB(reviews)         
end_time = time.time()    

execution_time = end_time - start_time
print("Время выполнения функции:", execution_time, "секунд")
mean_rating_newB(reviews_2010)

Время выполнения функции: 0.0 секунд


4.4544402182900615

In [22]:
lp = LineProfiler()
lp_wrapper_A = lp(mean_rating_A)
lp_wrapper_B = lp(mean_rating_newB)
lp_wrapper_C = lp(mean_rating_C)

lp_wrapper_A(reviews)
lp_wrapper_B(reviews_2010)
lp_wrapper_C(reviews_2010)
lp.print_stats()

Timer unit: 1e-07 s

Total time: 14.0556 s

Could not find file C:\Users\222844\AppData\Local\Temp/ipykernel_9980/1468465748.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           
     3         1          4.0      4.0      0.0  
     4         1          3.0      3.0      0.0  
     5    126696  124136039.0    979.8     88.3  
     6                                           
     7    114602   14991865.0    130.8     10.7  
     8     12094    1390983.0    115.0      1.0  
     9     12094      37593.0      3.1      0.0  
    10         1          7.0      7.0      0.0  

Total time: 0.0001186 s

Could not find file C:\Users\222844\AppData\Local\Temp/ipykernel_9980/3517253749.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continui

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [23]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count
get_word_reviews_count(reviews)

{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

In [24]:
profile = LineProfiler()
profile.add_function(get_word_reviews_count)
profile.enable()

get_word_reviews_count(reviews)

profile.disable()
profile.print_stats()

Timer unit: 1e-07 s

Total time: 45.1805 s

Could not find file C:\Users\222844\AppData\Local\Temp/ipykernel_9980/3588757971.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           
     2         1          7.0      7.0      0.0  
     3    126679  134977785.0   1065.5     29.9  
     4    126679   31922283.0    252.0      7.1  
     5    126679    4277274.0     33.8      0.9  
     6   6792010   16698669.0      2.5      3.7  
     7   6617066   23352644.0      3.5      5.2  
     8    174944     745313.0      4.3      0.2  
     9   6792010   29505537.0      4.3      6.5  
    10                                           
    11         1          5.0      5.0      0.0  
    12    126679  131283921.0   1036.4     29.1  
    13    126679   17362339.0    137.1      3.8  
    14    126679

В данном случае можно заметить, что второй цикл проходит по всему DataFrame, выполняя повторные разбиения текста на слова и проверку слов в словаре word_reviews каждый раз. Это неэффективно, особенно если DataFrame содержит много строк.

Для оптимизации можно использовать один цикл, чтобы построить словарь word_reviews и одновременно подсчитать количество отзывов для каждого слова

In [25]:
from collections import Counter
def get_word_reviews_count_optimized(df):    
    words_counts = Counter()  
    df_without_na = df.dropna(subset=["review"])  
    def update_word_counts(row):        
        if 'recipe_id' in row: 
            words = set(row)            
            words_counts.update(words)  
    pattern = r"[^A-Za-z\s]"  
    df_without_na['review'].str.replace(pattern, "").str.lower().str.split(" ").apply(update_word_counts)   
    return dict(words_counts.items())  

{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

In [26]:
profile = LineProfiler()
profile.add_function(get_word_reviews_count_new)
profile.enable()

get_word_reviews_count_new(reviews)

profile.disable()
profile.print_stats()

Timer unit: 1e-07 s

Total time: 27.3979 s

Could not find file C:\Users\222844\AppData\Local\Temp/ipykernel_9980/2735776033.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           
     2         1          4.0      4.0      0.0  
     3         1          3.0      3.0      0.0  
     4                                           
     5    126679  133125457.0   1050.9     48.6  
     6    126679   31368753.0    247.6     11.4  
     7    126679    4244442.0     33.5      1.5  
     8   6792010   17136083.0      2.5      6.3  
     9   6617066   22762524.0      3.4      8.3  
    10    174944     698737.0      4.0      0.3  
    11   6792010   30107635.0      4.4     11.0  
    12   6792010   34535058.0      5.1     12.6  
    13                                           
    14         1

Разбиение на слова выполняется только один раз перед циклом 
(счетчики уже подсчитываются внутри первого цикла).

Внутри первого цикла, после добавления recipe_id в word_reviews, также выполняется 
инкрементация счетчика word_reviews_count[word]. 
Мы используем метод .get(word, 0), чтобы получить текущее значение счетчика для слова word. 
Если слово уже присутствует в словаре word_reviews_count, мы увеличиваем счетчик на 1. 
Если слова еще нет в словаре, устанавливаем начальное значение 0 и затем инкрементируем его.

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [27]:
#Без использования векторизованных операций и методов массивов numpy и без использования numba:
def mape_a(df):
    ratings = [r for r in df['rating'] if r != 0]
    mean_rating = sum(ratings) / len(ratings)
    abs_diff = 0
    for r in ratings:
        abs_diff += abs(r - mean_rating)
    return abs_diff / len(ratings) * 100

start_time = time.time()
result = mape_a(reviews)
execution_time = time.time() - start_time

print("MAPE:", result)
print("Execution Time:", execution_time)

MAPE: 51.64167955430349
Execution Time: 0.024567604064941406


In [28]:
!pip install numba

  Using cached numpy-1.20.3-cp39-cp39-win_amd64.whl (13.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.0
    Uninstalling numpy-1.22.0:


ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'c:\\programdata\\anaconda3\\lib\\site-packages\\numpy-1.22.0.dist-info\\entry_points.txt'
Consider using the `--user` option or check the permissions.



In [36]:
#Без использования векторизованных операций и методов массивов numpy, но с использованием numba
import numba as nb

@nb.jit
def MAPE_B(df):
    sigma_diff = 0
    mean_rating = sum(df) / len(df)
    for r in df:  
        sigma_diff += abs((r - mean_rating)/r)
    return sigma_diff / len(df) * 100  


start_time = time.time()
REVIEWS=reviews[reviews['rating']!=0]['rating'].values
result = mape_b(REVIEWS)
execution_time = time.time() - start_time

print("MAPE:", result)
print("Execution Time:", execution_time)

ImportError: Numba needs NumPy 1.20 or less

In [37]:
#С использованием векторизованных операций и методов массивов numpy, но без использования numba:

import numpy as np

def mape_c(reviews):
    ratings = np.array([r for r in reviews['rating'] if r != 0])
    mean_rating = np.mean(ratings)
    abs_diff = np.sum(np.abs(ratings - mean_rating))
    return abs_diff / len(ratings) * 100

start_time = time.time()
result = mape_c(reviews)
execution_time = time.time() - start_time

print("MAPE:", result)
print("Execution Time:", execution_time)

MAPE: 51.64167955432964
Execution Time: 0.014000892639160156


In [38]:
# C использованием векторизованных операций и методов массивов numpy и numba
import numpy as np
from numba import jit

@jit
def mape_d(ratings):
    mean_rating = np.mean(ratings)
    sigma_diff = np.sum(np.abs((ratings - mean_rating)/ratings))
    return sigma_diff / len(ratings) * 100
ratings = np.array([r for r in reviews['rating'] if r != 0])
mape_d(ratings)

start_time = time.time()
ratings = np.array([r for r in reviews['rating'] if r != 0])
result = mape_d(ratings)
execution_time = time.time() - start_time

print("MAPE:", result)
print("Execution Time:", execution_time)


ImportError: Numba needs NumPy 1.20 or less

#### [версия 2]
* Уточнены формулировки задач 1, 3, 4

In [39]:
!pip install numpy==1.19.5
!pip install numba==0.54

  Using cached numpy-1.19.5-cp39-cp39-win_amd64.whl (13.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.0
    Uninstalling numpy-1.22.0:


ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'c:\\programdata\\anaconda3\\lib\\site-packages\\numpy-1.22.0.dist-info\\entry_points.txt'
Consider using the `--user` option or check the permissions.



  Using cached numba-0.54.0-cp39-cp39-win_amd64.whl (2.3 MB)
  Using cached numpy-1.20.3-cp39-cp39-win_amd64.whl (13.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.0
    Uninstalling numpy-1.22.0:


ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'c:\\programdata\\anaconda3\\lib\\site-packages\\numpy-1.22.0.dist-info\\entry_points.txt'
Consider using the `--user` option or check the permissions.

